In [22]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import time
import re
from collections import Counter
from collections import defaultdict
import operator
import math
import pickle

#DTW
import dtaidistance
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
from dtaidistance import clustering

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
#change r to 0 from 0.2
sys.path.append('C:\\Users\\camil\\Desktop\\animals_code\\entropy')
from entropy.entropy import sample_entropy
#from entropy.entropy import sample_entropy #https://github.com/raphaelvallat/entropy

In [24]:
from UTILS import time_series_henColumn_tsRow, ZoneVariable, HenVariable, DistributionEntropy, \
heatmap_duration_perzone_perhen, TimeSeriesPlot, entropy_compare_session
import config_mobility as config

In [25]:
path_extracted_data = config.path_extracted_data
id_run = config.id_run
nbr_sec = config.nbr_sec
path_extracted_visual = os.path.join(path_extracted_data, 'visual')

In [26]:
#day level
ts_name = 'time_serie_day'

## DTW per session

In [27]:
def DTWVariable(df_ts, config, window=120, value_delta=30, name='', save=True):

    ''' '''
    
    #start recording the time it last
    START_TIME = time.clock()

    #initialise variable
    path_extracted_data = config.path_extracted_data
    id_run = config.id_run
    dico_zone_order = config.dico_zone_order
    dico_S = {}

    #create path where to save if not existing yet
    path_ = os.path.join(path_extracted_data,'visual','DTW')
    #create a director if not existing
    if not os.path.exists(path_):
        os.makedirs(path_)
    
    #keep one value per value_delta value
    df_ts['nbr'] = range(df_ts.shape[0])
    df_ts['to_keep'] = df_ts['nbr'].map(lambda x: x%value_delta==0)
    df_ts = df_ts[df_ts['to_keep']]
    
    #keep only form 2h to 17h
    #df_ts['time'] = df_ts['Timestamp'].map(lambda x: dt.datetime.time(x)) #no need
    df_ts = df_ts[df_ts['hour'].isin([2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])]
        
    #from dataframe make a matrix with each row equal to one hen & transform the zone into integers
    li_hen = [i for i in df_ts.columns if i.startswith('hen_')]
    dico_zone_order_fl = {k:float(v) for k,v in dico_zone_order.items()}
    df_ts_dtw = df_ts.groupby(['day'])[li_hen].agg(lambda x: [dico_zone_order_fl[i] for i in x]).stack().reset_index()
    df_ts_dtw['name'] = df_ts_dtw.apply(lambda x: str(x['day']).split(' ')[0]+'_'+x['level_1'], axis=1)
    df_ts_dtw.rename(columns={0:'ts'}, inplace=True)
    li_ts_columns_name = df_ts_dtw['name'].tolist()
    M = np.array(df_ts_dtw.ts.values.tolist())
    print(M.shape)
            
    #hierarchical clustering to see nice picture
    # Custom Hierarchical clustering
    #print('model 1')
    #model1 = clustering.Hierarchical(dtw.distance_matrix_fast, {})
    #cluster_idx = model1.fit(M)
    # Augment Hierarchical object to keep track of the full tree
    #print('model 2')
    #model2 = clustering.HierarchicalTree(model1)
    #cluster_idx = model2.fit(M)
    #fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(22, 16))  # attention: 2 columns needed!
    #tree_plot = model2.plot(axes=ax, show_ts_label=li_hen, show_tr_label=True, ts_label_margin=-10, ts_left_margin=10)
    #plt.savefig(os.path.join(path_,id_run+'_'+name_+'_DTW_clustering1.png'), format='png')  
    # SciPy linkage clustering
    #print('model 3')
    #model3 = clustering.LinkageTree(dtw.distance_matrix_fast, {})
    #cluster_idx = model3.fit(M)
    #fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(100, 80)) 
    #tree_plot = model3.plot(axes=ax, show_tr_label=True, show_ts_label=li_ts_columns_name, ts_label_margin=-200)
    #plt.savefig(os.path.join(path_,id_run+'_'+name_+'_DTW_clustering2.png'), format='png')  

    #for each day and each two chickens draw the DTW matrix --> see path to compare them visually and record the similarity value to
    #plot the similarity evolution across each chicken
    dico_S = {}
    #for each day and each two chickens draw the DTW matrix --> see path to compare them visually and record the similarity value to
    #plot the similarity evolution across each chicken
    for i,h1 in enumerate(tqdm.tqdm(li_ts_columns_name)):
        li_h1 = M[i]
        li_h1 = np.array(li_h1, dtype=np.double)
        for j in range(i+1, len(li_ts_columns_name)):
            
            h2 = li_ts_columns_name[j]
            li_h2 = M[j]
            li_h2 = np.array(li_h2, dtype=np.double)
            d, paths = dtw.warping_paths(li_h1, li_h2, window=window, psi=2)
            dico_S[(h1,h2)] = d
            best_path = dtw.best_path(paths)
            dtwvis.plot_warpingpaths(li_h1, li_h2, paths, best_path);
            if save:
                plt.title (h1+'     '+h2)
                plt.savefig(os.path.join(path_,id_run+'_'+name_+'_DTW'+'_'+h1+'_'+h2+'.png'), format='png')  
            plt.close()
    if save:
        pickle.dump(dico_S, open(os.path.join(path_,id_run+'dico_S_'+name+'.pkl'), 'wb'))

    END_TIME = time.clock()
    print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))  

In [28]:
#open time series per session and compute the variables for each session (car time series make sence at session level), et en 
#plus des variables tel que running entropy over the whole session ateach last timestamp of each level make sence only at 
#session level
START_TIME = time.clock()
li_path = glob.glob(os.path.join(path_extracted_data, id_run+'_TimeSeries_*.csv'))
li_path = [i for i in li_path if '_hen_' not in i]
for path_ in tqdm.tqdm(li_path):
    
    #download time series associated to this session
    name_ = path_.split('_')[-1].split('.')[0]
    print('-------------------------------------------------------------------------', name_)
    df_ts = pd.read_csv(path_, sep=';', parse_dates=['Timestamp', 'day']) 
    #break
    #dynamical time warping
    DTWVariable(df_ts, config, name=name_)
    
END_TIME = time.clock()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))  

  0%|                                                                                           | 0/22 [00:00<?, ?it/s]

------------------------------------------------------------------------- 10A
(294, 1800)



  1%|▊                                                                             | 3/294 [25:03<40:00:05, 494.87s/it]

MemoryError: 